In [1]:
from sqlalchemy import create_engine, text, inspect, func
import sqlite3

# ORM imports
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, Boolean
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

# API
import requests
import json

# Data Science and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import datetime as dt

## CSV Imports

In [2]:
#CSV import for circuits table
filepath = "../Datasets/circuits.csv"
df_circuits = pd.read_csv(filepath, encoding="UTF-8")
df_circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [3]:
#CSV import for drivers table
filepath2 = "../Datasets/drivers.csv"
df_drivers = pd.read_csv(filepath2, encoding="UTF-8")
df_drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [4]:
#CSV import for standings table
filepath3 = "../Datasets/driver_standings.csv"
df_standings = pd.read_csv(filepath3, encoding="UTF-8")
df_standings.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


In [5]:
#CSV import for constructor table
filepath4 = "../Datasets/constructors.csv"
df_constructor = pd.read_csv(filepath4, encoding="UTF-8")
df_constructor.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [6]:
#CSV import for results table
filepath5 = "../Datasets/results.csv"
df_results = pd.read_csv(filepath5, encoding="UTF-8")
df_results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [7]:
#CSV import for races table
filepath6 = "../Datasets/races.csv"
df_races = pd.read_csv(filepath6, encoding="UTF-8")
df_races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


## Data Cleaning

In [8]:
#Remove url Column from df_circuits

columns_to_remove = ['url']

df_circuits.drop(columns_to_remove, axis=1, inplace=True)

df_circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130


In [9]:
# Consolidate United States and USA into 1 name
df_circuits["country"] = df_circuits.country.replace({"United States": "USA"})

In [10]:
# Remove url and code from df_drivers

columns_to_remove = ['url', 'code']

df_drivers.drop(columns_to_remove, axis=1, inplace=True)

df_drivers.head()

,driverId,driverRef,number,forename,surname,dob,nationality
0,1,hamilton,44,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,\N,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,6,Nico,Rosberg,1985-06-27,German
3,4,alonso,14,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,\N,Heikki,Kovalainen,1981-10-19,Finnish


In [11]:
df_drivers['dob']=pd.to_datetime(df_drivers['dob'])

df_drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     859 non-null    int64         
 1   driverRef    859 non-null    object        
 2   number       859 non-null    object        
 3   forename     859 non-null    object        
 4   surname      859 non-null    object        
 5   dob          859 non-null    datetime64[ns]
 6   nationality  859 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 47.1+ KB


In [12]:
#Remove extraneous columns from df_races

columns_to_remove = ['url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time','quali_date', 'quali_time', 'sprint_date', 'sprint_time']

df_races.drop(columns_to_remove, axis=1, inplace=True)

df_races.head()

,raceId,year,round,circuitId,name,date,time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00


In [13]:
#Remove extraneous columns from df_results

columns_to_remove = ['time', 'milliseconds', 'fastestLap', 'fastestLapTime','fastestLapSpeed']

df_results.drop(columns_to_remove, axis=1, inplace=True)

df_results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,rank,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,2,1
1,2,18,2,2,3,5,2,2,2,8.0,58,3,1
2,3,18,3,3,7,7,3,3,3,6.0,58,5,1
3,4,18,4,4,5,11,4,4,4,5.0,58,7,1
4,5,18,5,1,23,3,5,5,5,4.0,58,1,1


In [14]:
filt = df_results.position != "\\N"
df_results_clean = df_results.loc[filt]
df_results_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15591 entries, 0 to 26516
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   resultId       15591 non-null  int64  
 1   raceId         15591 non-null  int64  
 2   driverId       15591 non-null  int64  
 3   constructorId  15591 non-null  int64  
 4   number         15591 non-null  object 
 5   grid           15591 non-null  int64  
 6   position       15591 non-null  object 
 7   positionText   15591 non-null  object 
 8   positionOrder  15591 non-null  int64  
 9   points         15591 non-null  float64
 10  laps           15591 non-null  int64  
 11  rank           15591 non-null  object 
 12  statusId       15591 non-null  int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 1.7+ MB


In [15]:
# Checking value counts for df_standings['positionText']

df_standings['positionText'].value_counts()

positionText
1      1113
2      1113
3      1113
4      1113
5      1113
       ... 
105       2
107       1
D         1
108       1
106       1
Name: count, Length: 109, dtype: int64

In [16]:
df_constructor.drop('url', axis=1, inplace=True)
df_constructor.head()

,constructorId,constructorRef,name,nationality
0,1,mclaren,McLaren,British
1,2,bmw_sauber,BMW Sauber,German
2,3,williams,Williams,British
3,4,renault,Renault,French
4,5,toro_rosso,Toro Rosso,Italian


## Analysis

In [17]:
# What country has the most F1 circuits

df_circuits_country = df_circuits.groupby('country')

df_circuits_country['country'].count()

country
Argentina        1
Australia        2
Austria          2
Azerbaijan       1
Bahrain          1
Belgium          3
Brazil           2
Canada           3
China            1
France           7
Germany          3
Hungary          1
India            1
Italy            4
Japan            3
Korea            1
Malaysia         1
Mexico           1
Monaco           1
Morocco          1
Netherlands      1
Portugal         4
Qatar            1
Russia           1
Saudi Arabia     1
Singapore        1
South Africa     2
Spain            6
Sweden           1
Switzerland      1
Turkey           1
UAE              1
UK               4
USA             12
Name: country, dtype: int64

In [18]:
#  What nationality has the most drivers?

df_drivers_country = df_drivers.groupby('nationality')

df_drivers_country['nationality'].count()

nationality
American             158
American-Italian       1
Argentine             24
Argentine-Italian      1
Australian            18
Austrian              15
Belgian               23
Brazilian             32
British              166
Canadian              14
Chilean                1
Chinese                1
Colombian              3
Czech                  1
Danish                 5
Dutch                 18
East German            3
Finnish                9
French                73
German                50
Hungarian              1
Indian                 2
Indonesian             1
Irish                  5
Italian               99
Japanese              20
Liechtensteiner        1
Malaysian              1
Mexican                6
Monegasque             4
New Zealander         10
Polish                 1
Portuguese             4
Rhodesian              4
Russian                4
South African         23
Spanish               15
Swedish               10
Swiss                 23
Thai         

## SQLite Create Tables

In [19]:
# Create engine using the `f1.sqlite` database file
engine = create_engine("sqlite:///f1.sqlite")

In [20]:
# Get schema for each table
# Gotten from: https://stackoverflow.com/a/31075679
print(pd.io.sql.get_schema(df_circuits.reset_index(), 'circuits'))
print(pd.io.sql.get_schema(df_drivers.reset_index(), 'drivers'))
print(pd.io.sql.get_schema(df_constructor.reset_index(), 'constructors'))
print(pd.io.sql.get_schema(df_races.reset_index(), 'races'))
print(pd.io.sql.get_schema(df_results.reset_index(), 'results'))

CREATE TABLE "circuits" (
"index" INTEGER,
  "circuitId" INTEGER,
  "circuitRef" TEXT,
  "name" TEXT,
  "location" TEXT,
  "country" TEXT,
  "lat" REAL,
  "lng" REAL,
  "alt" INTEGER
)
CREATE TABLE "drivers" (
"index" INTEGER,
  "driverId" INTEGER,
  "driverRef" TEXT,
  "number" TEXT,
  "forename" TEXT,
  "surname" TEXT,
  "dob" TIMESTAMP,
  "nationality" TEXT
)
CREATE TABLE "constructors" (
"index" INTEGER,
  "constructorId" INTEGER,
  "constructorRef" TEXT,
  "name" TEXT,
  "nationality" TEXT
)
CREATE TABLE "races" (
"index" INTEGER,
  "raceId" INTEGER,
  "year" INTEGER,
  "round" INTEGER,
  "circuitId" INTEGER,
  "name" TEXT,
  "date" TEXT,
  "time" TEXT
)
CREATE TABLE "results" (
"index" INTEGER,
  "resultId" INTEGER,
  "raceId" INTEGER,
  "driverId" INTEGER,
  "constructorId" INTEGER,
  "number" TEXT,
  "grid" INTEGER,
  "position" TEXT,
  "positionText" TEXT,
  "positionOrder" INTEGER,
  "points" REAL,
  "laps" INTEGER,
  "rank" TEXT,
  "statusId" INTEGER
)


In [21]:
# Connect to database with SQLite
# Creating the tables was done with help from: https://www.geeksforgeeks.org/python-sqlite-create-table/
conn = sqlite3.connect('f1.sqlite')
cursor = conn.cursor()

In [22]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS CIRCUITS")

# Create the circuits table
table1 = """ CREATE TABLE "circuits" (
  circuitId INTEGER,
  circuitRef TEXT,
  name TEXT,
  location TEXT,
  country TEXT,
  lat REAL,
  lng REAL,
  alt INTEGER,
  PRIMARY KEY(circuitId)
  );"""

cursor.execute(table1)
 
print("Table is Ready")

# Load the table
df_circuits.to_sql("circuits", con=engine, index=False, method="multi", if_exists='append')

Table is Ready


77

In [23]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS DRIVERS")

# Create the drivers table
table1 = """ CREATE TABLE "drivers" (
  "driverId" INTEGER PRIMARY KEY,
  "driverRef" TEXT,
  "number" TEXT,
  "forename" TEXT,
  "surname" TEXT,
  "dob" TIMESTAMP,
  "nationality" TEXT
);"""

cursor.execute(table1)
 
print("Table is Ready")

# Load the table
df_drivers.to_sql("drivers", con=engine, index=False, method="multi", if_exists='append')

Table is Ready


859

In [24]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS CONSTRUCTORS")

# Create the constructors table
table3 = """ 
CREATE TABLE "constructors" (
  "constructorId" INTEGER PRIMARY KEY,
  "constructorRef" TEXT,
  "name" TEXT,
  "nationality" TEXT
);"""

cursor.execute(table3)
 
print("Table is Ready")

# Load the table
df_constructor.to_sql("constructors", con=engine, index=False, method="multi", if_exists='append')

Table is Ready


212

In [25]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS RACES")

# Create the races table
table4 = """ 
CREATE TABLE "races" (
  "raceId" INTEGER PRIMARY KEY,
  "year" INTEGER,
  "round" INTEGER,
  "circuitId" INTEGER,
  "name" TEXT,
  "date" TEXT,
  "time" TEXT
);
"""

cursor.execute(table4)
 
print("Table is Ready")

# Load the table
df_races.to_sql("races", con=engine, index=False, method="multi", if_exists='append')

Table is Ready


1125

In [26]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS RESULTS")

# Create the results table
table5 = """ 
CREATE TABLE "results" (
  "resultId" INTEGER PRIMARY KEY,
  "raceId" INTEGER,
  "driverId" INTEGER,
  "constructorId" INTEGER,
  "number" TEXT,
  "grid" INTEGER,
  "position" TEXT,
  "positionText" TEXT,
  "positionOrder" INTEGER,
  "points" REAL,
  "laps" INTEGER,
  "rank" TEXT,
  "statusId" INTEGER
);
"""

cursor.execute(table5)
 
print("Table is Ready")

# Load the table
df_results_clean.to_sql("results", con=engine, index=False, method="multi", if_exists='append')

Table is Ready


15591

In [27]:
# Close the connection
conn.close()

In [28]:
# End the engine
engine.dispose()